In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from datetime import date

## r진짜 찐막 제발

In [22]:
# 1) main title -> click & title extracting
driver = webdriver.Chrome()
driver.get("https://www.gotquestions.org/Korean/")

a_tags=driver.find_elements(By.TAG_NAME,"a")
print(len(a_tags))

52


In [23]:
# 영어를 포함한 값 추출해보기 - test
final=pd.DataFrame(columns=["crawling_date","Question_KOR","Answer_KOR","URL_KOR","Question_ENG","Answer_ENG","URL_ENG"])

In [ ]:
for i in range(1, 52): # 첫 번째 클릭 (대주제)
    
    try :
        element = driver.find_element(By.XPATH, f'/html/body/main/section[1]/div/strong/a[{i}]') # 버튼의 xpath를 이용 (1, 2, 3, ...)
        driver.execute_script("arguments[0].click();", element)
        time.sleep(0.5)
    

    except NoSuchElementException:
        time.sleep(0.5)
        break


    for ii in range(1, 120): # 두 번째 클릭 (소주제) 

        # 소주제 개수는 대주제마다 달라서, range()로 일단 1부터 120까지 범위 정해놓고
        # 반복문을 실행할 수 없는 상황, 찾으려는 요소가 없어서 더 이상 작업할 수 없을 때 반복문 종료
        try:
            element = driver.find_element(By.XPATH, f'/html/body/main/section[1]/div/strong/a[{ii}]') # 버튼의 xpath를 이용 (1, 2, 3, ...)
            driver.execute_script("arguments[0].click();", element)
            time.sleep(0.5)
        
        # 만약 버튼이 없다면 초기 화면으로 돌아간 후 초기 반복문으로 이동
        except NoSuchElementException:
            driver.get("https://www.gotquestions.org/Korean/")
            time.sleep(0.5)
            break
            
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
            
        # crawling 하는 날짜 date 넣기
        crawling_date = date.today()
        # 제목(소주제)는 <h1></h1> 사이에 존재
        title_kor = soup.find('h1').get_text(strip = True)
        # 현재 url
        url_kor = driver.current_url
            
        # 답변 내용은 "답변"과 "English" 사이에 존재한다는 공통점
        content_str_kor = str(soup.get_text(separator="\n"))
        start_kor = content_str_kor.find('답변') + len('답변') # 답변의 시작 지점
        end_kor = content_str_kor.find('English') # 답변의 끝 지점
        answer_html_kor = content_str_kor[start_kor : end_kor]
        answer_kor = BeautifulSoup(answer_html_kor, 'html.parser').get_text("\n", strip=True) # <>와 같은 태그 제거하여 텍스트만 남기기

        # ----------- #

        # English 클릭해서 해당 내용 긁어오기
        element = driver.find_element(By.LINK_TEXT, "English")
        driver.execute_script("arguments[0].click();", element)
        time.sleep(0.5)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
            
        # 영어 내용 크롤링
        # 영어 제목(소주제) <h1></h1> 사이에 존재
        title_eng = soup.find('h1').get_text(strip = True)

        # 현재 url
        url_eng = driver.current_url
            
        # 답변 내용은 "Question"과 "Return to:" 사이에 존재한다는 공통점
        content_str_eng = str(soup.get_text(separator="\n"))
        start_eng = content_str_eng.find('Answer') + len('Answer') # 답변의 시작 지점
        end_eng = content_str_eng.find('Return to:') # 답변의 끝 지점
        answer_html_eng = content_str_eng[start_eng : end_eng]
        answer_eng = BeautifulSoup(answer_html_eng, 'html.parser').get_text("\n", strip=True) # <>와 같은 태그 제거하여 텍스트만 남기기

        
        # 파일 저장
        new_row = pd.DataFrame({"crawling_date":[crawling_date], 
                                "Question_KOR": [title_kor], "Answer_KOR": [answer_kor], "URL_KOR": [url_kor],
                                "Question_ENG": [title_eng], "Answer_ENG": [answer_eng], "URL_ENG": [url_eng]})
        final = pd.concat([final, new_row], ignore_index=True)
        
        driver.back()
        time.sleep(0.5)

        driver.back()
        time.sleep(0.5)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/main/section[1]/div/strong/a[30]"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001028aea54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x00000001028a6cf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x00000001023f2864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000102439410 cxxbridge1$string$len + 382736
4   chromedriver                        0x000000010247a480 cxxbridge1$string$len + 649088
5   chromedriver                        0x000000010242d7ec cxxbridge1$string$len + 334572
6   chromedriver                        0x0000000102873ccc cxxbridge1$str$ptr + 2562928
7   chromedriver           

In [26]:
final

,crawling_date,Question_KOR,Answer_KOR,URL_KOR,Question_ENG,Answer_ENG,URL_ENG
0,2025-04-25,영생을 얻으셨습니까? 예수 그리스도를 믿음으로 말미암아 영원한 생명을 소유하셨습니까?,"성경은 영생에 이르는 분명한 길을 제시합니다. 먼저, 우리는 하나님 앞에 죄인임을 ...",https://www.gotquestions.org/Korean/Korean-Got...,Got Eternal Life?,The Bible presents a clear path to eternal lif...,https://www.gotquestions.org/eternal-life.html
1,2025-04-25,용서를 받으셨습니까? 하나님께 죄 사함을 받는 방법은 무엇입니까?,"‘용서하다 ’라는 단어는 과거에 저지른 잘못을 덮어주고, 눈감아주며, 빚을 청산해...",https://www.gotquestions.org/Korean/Korean-Got...,Got Forgiveness?,"Acts 13:38\n declares, ""Therefore, my brothers...",https://www.gotquestions.org/got-forgiveness.html
2,2025-04-25,죄인의 기도란 무엇인가?,‘죄인의 기도(Sinner’s Prayer)’는 자신이 죄인이며 구주가 필요하다는 ...,https://www.gotquestions.org/Korean/Korean-Sin...,What is the sinner’s prayer?,The sinner’s prayer is a prayer a person prays...,https://www.gotquestions.org/sinners-prayer.html
3,2025-04-25,예수님만이 천국에 이르는 유일한 길인가?,그렇습니다. 예수님만이 천국에 이르는 유일한 길입니다. 이런 배타적인 표현은 포스...,https://www.gotquestions.org/Korean/Korean-Jes...,Is Jesus the only way to heaven?,"Yes, Jesus is the only way to heaven. Such an ...",https://www.gotquestions.org/Jesus-only-way.html
4,2025-04-25,죽은 뒤 천국에 간다는 것을 어떻게 확신할 수 있는가?,여러분은 자신이 영생을 얻었고 죽은 뒤 천국에 갈 것이라는 사실을 확신하십니까? 하...,https://www.gotquestions.org/Korean/Korean-Kno...,How can I know for sure that I will go to heav...,Do you know for certain that you have eternal ...,https://www.gotquestions.org/know-sure-Heaven....
...,...,...,...,...,...,...,...
1435,2025-04-25,아볼로는 누구입니까?,"아볼로는 전도자이자 변증가였고, 교회 지도자이며, 사도 바울의 친구였습니다. 아볼로...",https://www.gotquestions.org/Korean/Korean-who...,Who was Apollos?,"Apollos was an evangelist, apologist, church l...",https://www.gotquestions.org/who-Apollos.html
1436,2025-04-25,바나바의 삶을 통해 우리는 무엇을 배울 수 있습니까?,"사도행전에는 사도들이 바나바라고 불렀던, 요셉이라는 이름을 가진 구브로 출신의 레위...",https://www.gotquestions.org/Korean/Korean-lif...,Who was Barnabas in the Bible?,"In the book of Acts, we find a Levite from \nC...",https://www.gotquestions.org/life-Barnabas.html
1437,2025-04-25,에스더의 삶을 통해 우리는 무엇을 배울 수 있습니까?,에스더는 페르시아의 왕비가 되어 유대 민족을 전멸하기 위해 온 민족을 죽이려는 음모...,https://www.gotquestions.org/Korean/Korean-lif...,Who was Esther in the Bible?,Esther is the Jewish maiden who became queen o...,https://www.gotquestions.org/life-Esther.html
1438,2025-04-25,에스겔의 삶을 통해 우리는 무엇을 배울 수 있습니까?,에스겔의 이름은 '하나님께서 강하게 하심'이라는 뜻을 가지고 있습니다. 그는 예루살...,https://www.gotquestions.org/Korean/Korean-lif...,Who was Ezekiel in the Bible?,"Ezekiel, whose name means “strengthened by God...",https://www.gotquestions.org/life-Ezekiel.html


In [27]:
final.to_excel(f"/Users/haley/Desktop/2025-1/DS1/code/preprocessing/GodQuestions/GodQuestions_test_Kor_{crawling_date}.xlsx",index=False)